In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

Adds adjacency to RSNG by routing along atlas

In [2]:
atlas = src.graph.graph_from_json('atlas.json')

In [3]:
atlas.number_of_nodes(), atlas.number_of_edges()

(59621, 63677)

In [4]:
graph_empty = src.graph.graph_from_json('Outputs/graph_empty.json')

In [5]:
graph_empty.number_of_nodes(), graph_empty.number_of_edges()

(1727, 0)

In [6]:
# Adding roadnum for destinations

graph_to_atlas, atlas_to_graph = src.adjacency.node_assignment(graph_empty, atlas)

for graph_node, atlas_node in graph_to_atlas.items():

    graph_empty._node[graph_node]['roadnum'] = atlas._node[atlas_node]['roadnum']

In [13]:
deep_reload(src)

objective = src.adjacency.Graph_From_Atlas(
    limits = [np.inf, 786e3, np.inf]
)

graph = src.adjacency.adjacency(
    atlas,
    graph_empty,
    objective = objective,
)

 [------>] (1727/1727) 100.00%, 0.72 it/s, 2407.13 s elapsed, 0.00 s remaining

In [14]:
src.graph.graph_to_json(graph, 'Outputs/graph_full.json')

In [15]:
graph.number_of_nodes(), graph.number_of_edges()

(1727, 1369614)

In [2]:
'''
Loading the graph with adjacency
'''

graph = src.graph.graph_from_json('Outputs/graph_full.json')
graph = src.routing.edge_types(graph)

In [3]:
for source, node in graph._node.items():

    if 'pop' in node:

        node['population'] = node['pop']
        node.pop('pop')

    if 'station' not in source:

        node['type'] = 'place'

In [5]:
'''
Removing non-corridor stations
'''
roadnums = []

for source, node in graph._node.items():
    
    roadnums.append(node['roadnum'])

unique_roadnum, unique_roadnum_count = np.unique(roadnums, return_counts = True)

roadnums_keep = unique_roadnum[(unique_roadnum_count >= 10) & (unique_roadnum != 'nan')]
roadnums_keep = np.append(roadnums_keep, 'U395')

corridor_roadnum = []

corridor_roadnum.extend([r for r in unique_roadnum if 'I' in r])
corridor_roadnum.extend([r for r in unique_roadnum if 'U' in r])
corridor_roadnum.extend(['S1', 'S41', 'S99'])

roadnums_keep = corridor_roadnum

nodes_keep = []

for source, node in graph._node.items():

    roadnum = node['roadnum']

    if node['type'] == 'place':

        nodes_keep.append(source)
    
    else:

        if roadnum in roadnums_keep:
    
            nodes_keep.append(source)

roadnums_keep = np.append(roadnums_keep, 'U395')

sng_corridor = src.graph.subgraph(graph, nodes_keep)

In [9]:
sng_corridor.number_of_nodes(), sng_corridor.number_of_edges()

(678, 202673)

In [10]:
u, c = np.unique(
    [n.get('network', '') for n in sng_corridor._node.values()], return_counts = True
)

other_networks = u[(c >= 50) & (u != 'Tesla') & (u != 'Non-Networked')]
other_networks = u

In [11]:
'''
Creating Tesla and non-tesla SNGs
'''

tesla = []
other = []
combined = []
rivian = []

for source, node in sng_corridor._node.items():

    if node['type'] == 'place':

        tesla.append(source)
        other.append(source)
        combined.append(source)
        rivian.append(source)

    else:

        include = False

        if node['network'] == 'Tesla':

            tesla.append(source)
            combined.append(source)

        else:

            other.append(source)
            
        if node['network'] in other_networks:

            combined.append(source)

        if node['network'] in ['RIVIAN_ADVENTURE']:

            rivian.append(source)

sng_tesla = src.graph.subgraph(graph, tesla)
sng_other = src.graph.subgraph(graph, other)
sng_combined = src.graph.subgraph(graph, combined)
sng_rivian = src.graph.subgraph(graph, rivian)

In [12]:
sng_combined.number_of_nodes(), sng_tesla.number_of_nodes(), sng_other.number_of_nodes(), sng_rivian.number_of_nodes()

(678, 218, 498, 50)

In [13]:
src.graph.graph_to_json(sng_combined, 'Outputs/sng_combined.json')
src.graph.graph_to_json(sng_tesla, 'Outputs/sng_tesla.json')
src.graph.graph_to_json(sng_other, 'Outputs/sng_other.json')
src.graph.graph_to_json(sng_rivian, 'Outputs/sng_sng_rivian.json')